# QC-Py-14 - Portfolio Construction et Execution Models

> **Transformer les Insights en positions optimales et les executer efficacement**
> Duree: 90 minutes | Niveau: Intermediaire-Avance | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Comprendre le **role du Portfolio Construction Model** dans le Framework
2. Utiliser les modeles **EqualWeighting** et **InsightWeighting**
3. Implementer un **MeanVarianceOptimization** Portfolio Model
4. Creer un **Custom Portfolio Construction Model** (Risk Parity)
5. Maitriser les **Execution Models** (Immediate, VWAP)
6. Implementer un **Smart Execution Model** avec controle du slippage
7. Utiliser les **Risk Management Models** integres et composites
8. Construire une **strategie Framework complete** avec tous les composants

## Prerequis

- Notebooks QC-Py-01 a QC-Py-12 completes
- Comprehension du Framework QuantConnect (Alpha, Universe Selection)
- Notions de base en optimisation de portefeuille

## Structure du Notebook

| Partie | Sujet | Duree |
|--------|-------|-------|
| 1 | Role du Portfolio Construction Model | 10 min |
| 2 | EqualWeightingPortfolioConstructionModel | 8 min |
| 3 | InsightWeightingPortfolioConstructionModel | 10 min |
| 4 | MeanVarianceOptimizationPortfolioConstructionModel | 12 min |
| 5 | Custom Portfolio Construction Model (Risk Parity) | 15 min |
| 6 | Execution Models (Immediate, VWAP) | 18 min |
| 7 | Custom Smart Execution Model | 12 min |
| 8 | Risk Management Models | 15 min |
| 9 | Strategie Framework Complete | 20 min |

---

## Introduction : Le Framework QuantConnect

Le **Framework QuantConnect** decompose une strategie de trading en composants modulaires. Ce notebook se concentre sur les **3 derniers etages** de la pipeline :

```
Pipeline du Framework QuantConnect:

1. Universe Selection Model
   |
   v
2. Alpha Model (genere des Insights)
   |
   v
3. PORTFOLIO CONSTRUCTION MODEL  <-- Ce notebook
   |  Transforme Insights -> PortfolioTargets
   v
4. RISK MANAGEMENT MODEL         <-- Ce notebook
   |  Ajuste les targets selon les risques
   v
5. EXECUTION MODEL               <-- Ce notebook
      Execute les targets (ordres)
```

### Pourquoi separer ces composants ?

| Composant | Responsabilite | Avantage de la separation |
|-----------|----------------|---------------------------|
| **Alpha Model** | Generer des signaux (Insights) | Focus sur la prediction |
| **Portfolio Construction** | Allocation des poids | Testable independamment |
| **Risk Management** | Contraintes de risque | Regle de risque reutilisable |
| **Execution** | Placement des ordres | Optimisation des couts |

In [ ]:
# Imports QuantConnect
from AlgorithmImports import *

# Imports pour analyse
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

print("Imports reussis")

---

## Partie 1 : Role du Portfolio Construction Model (10 min)

### Qu'est-ce qu'un Portfolio Construction Model ?

Le **Portfolio Construction Model** recoit les **Insights** generes par l'Alpha Model et les transforme en **Portfolio Targets** (allocations cibles).

### Flux de donnees

```
Alpha Model                    Portfolio Construction          Execution
    |                                  |
    +-- Insight(AAPL, Up, 0.8)  -->   |                         |
    +-- Insight(MSFT, Up, 0.6)  -->   +-- PortfolioTarget(AAPL, 0.40)
    +-- Insight(TSLA, Down, 0.9) -->  +-- PortfolioTarget(MSFT, 0.30)
                                      +-- PortfolioTarget(TSLA, -0.20)
                                            |
                                            v
                                      Ordres d'achat/vente
```

### Proprietes d'un Insight

| Propriete | Type | Description |
|-----------|------|-------------|
| `Symbol` | Symbol | L'actif concerne |
| `Direction` | InsightDirection | Up, Down, ou Flat |
| `Magnitude` | float | Force attendue du mouvement |
| `Confidence` | float | Niveau de confiance (0-1) |
| `Period` | TimeSpan | Duree de validite de l'insight |
| `Weight` | float | Poids optionnel suggere |

### Proprietes d'un PortfolioTarget

| Propriete | Type | Description |
|-----------|------|-------------|
| `Symbol` | Symbol | L'actif concerne |
| `Quantity` | decimal | Nombre d'actions cibles |
| `Tag` | string | Commentaire optionnel |

In [ ]:
# Structure de base d'un Portfolio Construction Model
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class BasicPortfolioConstructionModel(PortfolioConstructionModel):
    """
    Template de base pour un Portfolio Construction Model.
    Herite de PortfolioConstructionModel et implemente CreateTargets.
    """
    
    def __init__(self):
        """Initialisation du modele."""
        pass
    
    def CreateTargets(self, algorithm, insights):
        """
        Methode principale : transforme les Insights en PortfolioTargets.
        
        Args:
            algorithm: QCAlgorithm - acces aux donnees et portfolio
            insights: List[Insight] - insights actifs generes par l'Alpha Model
        
        Returns:
            List[PortfolioTarget] - allocations cibles
        """
        targets = []
        
        for insight in insights:
            # Logique de construction de portfolio
            # ...
            pass
        
        return targets
    
    def OnSecuritiesChanged(self, algorithm, changes):
        """
        Appele quand l'univers change.
        Permet de nettoyer les positions sur les titres retires.
        """
        for security in changes.RemovedSecurities:
            # Optionnel: fermer les positions retirees
            pass

print("Template BasicPortfolioConstructionModel defini")
print("\nMethodes principales:")
print("  - __init__(): Initialisation")
print("  - CreateTargets(): Insights -> PortfolioTargets")
print("  - OnSecuritiesChanged(): Gestion des changements d'univers")

### Les Portfolio Construction Models integres

QuantConnect fournit plusieurs modeles pre-implementes :

| Modele | Description | Cas d'usage |
|--------|-------------|-------------|
| `EqualWeightingPortfolioConstructionModel` | Meme poids pour tous les insights actifs | Simple, pas de biais |
| `InsightWeightingPortfolioConstructionModel` | Ponderation par magnitude/confidence | Exploiter la conviction |
| `MeanVarianceOptimizationPortfolioConstructionModel` | Optimisation Markowitz | Maximum Sharpe |
| `BlackLittermanOptimizationPortfolioConstructionModel` | Black-Litterman avec views | Combine marche + alpha |
| `NullPortfolioConstructionModel` | Ne fait rien | Desactive le composant |

---

## Partie 2 : EqualWeightingPortfolioConstructionModel (8 min)

### Le modele le plus simple

Le **EqualWeightingPortfolioConstructionModel** attribue un poids egal a tous les actifs avec des insights actifs. C'est l'equivalent d'un portefeuille equipondere.

### Formule

$$w_i = \frac{direction_i}{\sum |direction_j|}$$

Ou :
- $w_i$ = poids de l'actif i
- $direction_i$ = +1 (Up), -1 (Down), 0 (Flat)

### Exemple

Avec 4 insights actifs (3 Up, 1 Down) :
- AAPL (Up): +25%
- MSFT (Up): +25%
- GOOGL (Up): +25%
- TSLA (Down): -25%

In [ ]:
# Utilisation de EqualWeightingPortfolioConstructionModel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class EqualWeightExample(QCAlgorithm):
    """
    Exemple d'utilisation du EqualWeightingPortfolioConstructionModel.
    Tous les actifs avec insights actifs ont le meme poids.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter les actifs
        self.AddEquity("SPY", Resolution.Daily)
        self.AddEquity("QQQ", Resolution.Daily)
        self.AddEquity("IWM", Resolution.Daily)
        self.AddEquity("TLT", Resolution.Daily)
        
        # === PORTFOLIO CONSTRUCTION MODEL ===
        # EqualWeighting : tous les insights actifs ont le meme poids
        self.SetPortfolioConstruction(
            EqualWeightingPortfolioConstructionModel()
        )
        
        # Execution immediate
        self.SetExecution(ImmediateExecutionModel())
        
        # Alpha Model simple (RSI)
        self.SetAlpha(RsiAlphaModel())
        
        self.Log("EqualWeightingPortfolioConstructionModel configure")
    
    def OnData(self, data):
        pass

print("EqualWeightExample defini")
print("\nCaracteristiques:")
print("  - Poids identique pour tous les insights actifs")
print("  - Direction Up -> poids positif")
print("  - Direction Down -> poids negatif (short)")
print("  - Simple et sans biais")

In [ ]:
# EqualWeighting avec options de rebalancement
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class EqualWeightWithRebalance(QCAlgorithm):
    """
    EqualWeighting avec rebalancement periodique.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs
        symbols = ["SPY", "QQQ", "IWM", "TLT", "GLD"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        # Portfolio Construction avec rebalancement hebdomadaire
        self.SetPortfolioConstruction(
            EqualWeightingPortfolioConstructionModel(
                rebalance=Resolution.Weekly  # Rebalancer chaque semaine
            )
        )
        
        # Alternatives de rebalancement:
        # rebalance=Resolution.Daily     # Quotidien
        # rebalance=Resolution.Monthly   # Mensuel
        # rebalance=Expiry.EndOfMonth    # Fin de mois
        # rebalance=lambda time: time.day == 1  # Premier du mois
        
        self.SetExecution(ImmediateExecutionModel())
        self.SetAlpha(ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=7)))
        
        self.Log("EqualWeighting with weekly rebalancing configured")

print("EqualWeightWithRebalance defini")
print("\nOptions de rebalancement:")
print("  - Resolution.Daily")
print("  - Resolution.Weekly")
print("  - Resolution.Monthly")
print("  - Expiry.EndOfMonth")
print("  - Custom lambda function")

### Avantages et limites de EqualWeighting

| Avantages | Limites |
|-----------|---------||
| Simple a comprendre | Ignore la conviction (confidence) |
| Pas de biais | Ignore la magnitude attendue |
| Robuste | Peut surponderer des signaux faibles |
| Diversifie naturellement | Pas d'optimisation du risque |

---

## Partie 3 : InsightWeightingPortfolioConstructionModel (10 min)

### Ponderation par l'Insight

Le **InsightWeightingPortfolioConstructionModel** utilise les proprietes des Insights pour determiner les poids :

- **Magnitude** : Force attendue du mouvement
- **Confidence** : Niveau de conviction

### Formule

$$w_i = \frac{direction_i \times magnitude_i \times confidence_i}{\sum_j |direction_j \times magnitude_j \times confidence_j|}$$

### Exemple

| Symbol | Direction | Magnitude | Confidence | Score | Poids |
|--------|-----------|-----------|------------|-------|-------|
| AAPL | Up (+1) | 0.10 | 0.80 | 0.080 | 40% |
| MSFT | Up (+1) | 0.05 | 0.60 | 0.030 | 15% |
| GOOGL | Up (+1) | 0.08 | 0.70 | 0.056 | 28% |
| TSLA | Down (-1) | 0.06 | 0.55 | -0.033 | -17% |

In [ ]:
# InsightWeightingPortfolioConstructionModel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class InsightWeightExample(QCAlgorithm):
    """
    Utilisation du InsightWeightingPortfolioConstructionModel.
    Ponderation basee sur la magnitude et la confidence des insights.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs
        self.AddEquity("SPY", Resolution.Daily)
        self.AddEquity("QQQ", Resolution.Daily)
        self.AddEquity("IWM", Resolution.Daily)
        self.AddEquity("TLT", Resolution.Daily)
        
        # === PORTFOLIO CONSTRUCTION MODEL ===
        # InsightWeighting : poids proportionnels a magnitude * confidence
        self.SetPortfolioConstruction(
            InsightWeightingPortfolioConstructionModel()
        )
        
        self.SetExecution(ImmediateExecutionModel())
        
        # Alpha Model personnalise qui genere des insights avec magnitude/confidence
        self.SetAlpha(MomentumAlphaModel())
        
        self.Log("InsightWeightingPortfolioConstructionModel configure")

print("InsightWeightExample defini")
print("\nCaracteristiques:")
print("  - Poids = Direction * Magnitude * Confidence")
print("  - Insights avec forte conviction -> poids plus eleves")
print("  - Permet d'exploiter la qualite des signaux")

In [ ]:
# Alpha Model personnalise avec magnitude et confidence
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class MomentumAlphaModel(AlphaModel):
    """
    Alpha Model qui genere des insights avec magnitude et confidence
    basees sur le momentum.
    """
    
    def __init__(self, lookback=20):
        self.lookback = lookback
        self.securities = []
    
    def Update(self, algorithm, data):
        insights = []
        
        for symbol in self.securities:
            if not data.ContainsKey(symbol):
                continue
            
            # Recuperer historique
            history = algorithm.History(symbol, self.lookback, Resolution.Daily)
            if history.empty or len(history) < self.lookback:
                continue
            
            # Calculer momentum
            returns = history['close'].pct_change().dropna()
            momentum = returns.mean()
            volatility = returns.std()
            
            if volatility == 0:
                continue
            
            # Direction basee sur le momentum
            direction = InsightDirection.Up if momentum > 0 else InsightDirection.Down
            
            # Magnitude = momentum absolu (normalise)
            magnitude = min(abs(momentum) * 10, 0.2)  # Cap a 20%
            
            # Confidence = Sharpe ratio normalise (0 a 1)
            sharpe = momentum / volatility if volatility > 0 else 0
            confidence = min(max(abs(sharpe) / 3, 0.1), 1.0)  # Entre 0.1 et 1.0
            
            # Generer l'insight
            insight = Insight.Price(
                symbol,
                timedelta(days=5),
                direction,
                magnitude,
                confidence
            )
            insights.append(insight)
            
            algorithm.Debug(f"{symbol.Value}: mom={momentum:.4f}, dir={direction}, mag={magnitude:.3f}, conf={confidence:.2f}")
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.AddedSecurities:
            self.securities.append(security.Symbol)
        for security in changes.RemovedSecurities:
            if security.Symbol in self.securities:
                self.securities.remove(security.Symbol)

print("MomentumAlphaModel defini")
print("\nGenere des insights avec:")
print("  - Direction: basee sur le signe du momentum")
print("  - Magnitude: momentum absolu normalise")
print("  - Confidence: Sharpe ratio normalise")

### Quand utiliser InsightWeighting ?

| Situation | Recommandation |
|-----------|----------------|
| Alpha Model genere magnitude/confidence | InsightWeighting |
| Signaux tous equivalents | EqualWeighting |
| Modele ML avec probabilites | InsightWeighting |
| Signaux discretionnaires | EqualWeighting |

---

## Partie 4 : MeanVarianceOptimizationPortfolioConstructionModel (12 min)

### Optimisation de Markowitz

Le **MeanVarianceOptimizationPortfolioConstructionModel** utilise la theorie moderne du portefeuille (MPT) pour optimiser l'allocation :

$$\max_w \left( w^T \mu - \frac{\lambda}{2} w^T \Sigma w \right)$$

Ou :
- $w$ = vecteur des poids
- $\mu$ = rendements attendus (derives des insights)
- $\Sigma$ = matrice de covariance
- $\lambda$ = aversion au risque

### Options disponibles

| Parametre | Options | Description |
|-----------|---------|-------------|
| `rebalancingPeriod` | Resolution/Expiry | Frequence de rebalancement |
| `portfolioBias` | Long, Short, LongShort | Contrainte de direction |
| `lookback` | int | Jours pour estimer la covariance |
| `resolution` | Resolution | Resolution des donnees historiques |

In [ ]:
# MeanVarianceOptimizationPortfolioConstructionModel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from Portfolio.MeanVarianceOptimizationPortfolioConstructionModel import *

class MeanVarianceOptimizationExample(QCAlgorithm):
    """
    Utilisation du MeanVarianceOptimization pour optimiser le portfolio.
    Maximise le Sharpe Ratio sous contraintes.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs diversifies
        symbols = ["SPY", "QQQ", "IWM", "TLT", "GLD", "EFA", "EEM"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        # === MEAN-VARIANCE OPTIMIZATION ===
        self.SetPortfolioConstruction(
            MeanVarianceOptimizationPortfolioConstructionModel(
                rebalancingPeriod=Expiry.EndOfMonth,  # Rebalancement mensuel
                portfolioBias=PortfolioBias.Long,     # Long only
                lookback=60,                           # 60 jours d'historique
                resolution=Resolution.Daily
            )
        )
        
        # Execution
        self.SetExecution(ImmediateExecutionModel())
        
        # Alpha Model simple
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=30))
        )
        
        self.Log("MeanVarianceOptimizationPortfolioConstructionModel configure")
        self.Log("  Rebalancement: Fin de mois")
        self.Log("  Biais: Long only")
        self.Log("  Lookback: 60 jours")

print("MeanVarianceOptimizationExample defini")
print("\nParametres:")
print("  - rebalancingPeriod: Expiry.EndOfMonth")
print("  - portfolioBias: PortfolioBias.Long")
print("  - lookback: 60 jours")

In [ ]:
# Options de PortfolioBias
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from Portfolio.MeanVarianceOptimizationPortfolioConstructionModel import *

class MeanVarianceLongShort(QCAlgorithm):
    """
    MeanVariance avec positions Long et Short.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        symbols = ["SPY", "QQQ", "IWM", "TLT", "GLD"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        # === LONG-SHORT OPTIMIZATION ===
        self.SetPortfolioConstruction(
            MeanVarianceOptimizationPortfolioConstructionModel(
                rebalancingPeriod=Resolution.Weekly,
                portfolioBias=PortfolioBias.LongShort,  # Autorise les shorts
                lookback=90,
                resolution=Resolution.Daily
            )
        )
        
        self.SetExecution(ImmediateExecutionModel())
        self.SetAlpha(MomentumAlphaModel())
        
        self.Log("MeanVariance Long-Short configure")

print("MeanVarianceLongShort defini")
print("\nOptions PortfolioBias:")
print("  - PortfolioBias.Long: Positions longues uniquement")
print("  - PortfolioBias.Short: Positions courtes uniquement")
print("  - PortfolioBias.LongShort: Les deux")

### Avantages et limites de MeanVariance

| Avantages | Limites |
|-----------|---------||
| Optimise le ratio risque/rendement | Sensible aux estimations de covariance |
| Base theorique solide (MPT) | Peut generer des positions concentrees |
| Prend en compte les correlations | Plus lent (optimisation numerique) |
| Configurable (Long/Short) | Necessite suffisamment d'historique |

---

## Partie 5 : Custom Portfolio Construction Model - Risk Parity (15 min)

### Qu'est-ce que Risk Parity ?

**Risk Parity** est une approche d'allocation qui egalise la contribution au risque de chaque actif plutot que les montants investis.

### Principe

- **Equal Weight** : Memes montants en dollars -> risque domine par les actifs volatils
- **Risk Parity** : Meme contribution au risque -> poids inverses a la volatilite

### Formule simplifiee (Inverse Volatility)

$$w_i = \frac{1/\sigma_i}{\sum_j 1/\sigma_j}$$

Ou $\sigma_i$ est la volatilite de l'actif i.

### Exemple

| Actif | Volatilite | 1/Vol | Poids |
|-------|------------|-------|-------|
| Actions (SPY) | 20% | 5 | 27.8% |
| Obligations (TLT) | 10% | 10 | 55.6% |
| Or (GLD) | 15% | 6.67 | 16.7% |

In [ ]:
# Custom Risk Parity Portfolio Construction Model
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
import numpy as np

class RiskParityPortfolioConstruction(PortfolioConstructionModel):
    """
    Portfolio Construction Model basé sur Risk Parity.
    Alloue les poids inversement proportionnels a la volatilite.
    """
    
    def __init__(self, rebalance_period=30, lookback=60):
        """
        Args:
            rebalance_period: Jours entre chaque rebalancement
            lookback: Jours d'historique pour calculer la volatilite
        """
        self.rebalance_period = rebalance_period
        self.lookback = lookback
        self.last_rebalance = datetime.min
    
    def CreateTargets(self, algorithm, insights):
        """
        Cree les PortfolioTargets basés sur Risk Parity.
        
        Args:
            algorithm: QCAlgorithm
            insights: Liste des insights actifs
        
        Returns:
            Liste de PortfolioTarget
        """
        targets = []
        
        # Verifier si c'est le moment de rebalancer
        if (algorithm.Time - self.last_rebalance).days < self.rebalance_period:
            return targets
        
        self.last_rebalance = algorithm.Time
        
        # Filtrer les insights actifs avec direction Up ou Down
        active_insights = [i for i in insights 
                          if i.Direction != InsightDirection.Flat]
        
        if len(active_insights) == 0:
            return targets
        
        # Calculer les volatilites pour chaque symbole
        volatilities = {}
        
        for insight in active_insights:
            symbol = insight.Symbol
            
            # Recuperer historique
            history = algorithm.History(symbol, self.lookback, Resolution.Daily)
            
            if history.empty or len(history) < self.lookback * 0.8:
                continue
            
            try:
                # Calculer returns et volatilite annualisee
                returns = history['close'].pct_change().dropna()
                vol = returns.std() * np.sqrt(252)
                
                if vol > 0:
                    volatilities[symbol] = vol
                    algorithm.Debug(f"{symbol.Value}: Vol = {vol:.2%}")
            except:
                continue
        
        if len(volatilities) == 0:
            return targets
        
        # Calculer poids par volatilite inverse (Risk Parity)
        inv_vols = {s: 1.0 / v for s, v in volatilities.items()}
        total_inv_vol = sum(inv_vols.values())
        
        # Creer les targets
        for insight in active_insights:
            symbol = insight.Symbol
            
            if symbol not in inv_vols:
                continue
            
            # Poids de base (Risk Parity)
            base_weight = inv_vols[symbol] / total_inv_vol
            
            # Ajuster selon la direction de l'insight
            direction = 1 if insight.Direction == InsightDirection.Up else -1
            final_weight = direction * base_weight
            
            # Creer le target
            targets.append(PortfolioTarget(symbol, final_weight))
            algorithm.Debug(f"Target {symbol.Value}: {final_weight:.2%}")
        
        algorithm.Debug(f"Risk Parity: {len(targets)} targets generes")
        return targets
    
    def OnSecuritiesChanged(self, algorithm, changes):
        """Gerer les changements d'univers."""
        for security in changes.RemovedSecurities:
            if security.Invested:
                algorithm.Liquidate(security.Symbol)

print("RiskParityPortfolioConstruction defini")
print("\nCaracteristiques:")
print("  - Poids = 1/Volatilite (normalise)")
print("  - Actifs moins volatils -> poids plus eleves")
print("  - Contribution au risque egale pour tous les actifs")

In [ ]:
# Utilisation du Risk Parity Portfolio Construction Model
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class RiskParityAlgorithm(QCAlgorithm):
    """
    Strategie utilisant Risk Parity pour l'allocation.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs avec volatilites differentes
        symbols = ["SPY", "QQQ", "TLT", "GLD", "IEF"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        # === RISK PARITY PORTFOLIO CONSTRUCTION ===
        self.SetPortfolioConstruction(
            RiskParityPortfolioConstruction(
                rebalance_period=30,  # Mensuel
                lookback=60            # 60 jours d'historique
            )
        )
        
        self.SetExecution(ImmediateExecutionModel())
        
        # Alpha: Long sur tous les actifs
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=30))
        )
        
        self.Log("Risk Parity Algorithm initialized")

print("RiskParityAlgorithm defini")
print("\nAllocation typique Risk Parity:")
print("  - Obligations (faible vol) -> poids eleves (~40-50%)")
print("  - Or (vol moyenne) -> poids moyens (~15-20%)")
print("  - Actions (haute vol) -> poids faibles (~20-30%)")

---

## Partie 6 : Execution Models (18 min)

### Role de l'Execution Model

L'**Execution Model** recoit les **PortfolioTargets** et les transforme en **ordres d'achat/vente**. Il gere :

- Le **type d'ordre** (Market, Limit, Stop)
- Le **timing** de l'execution
- La **gestion du slippage**
- Le **splitting** des gros ordres

### Execution Models integres

| Modele | Description | Cas d'usage |
|--------|-------------|-------------|
| `ImmediateExecutionModel` | Ordres Market immediats | Simple, liquid markets |
| `VolumeWeightedAveragePriceExecutionModel` | VWAP execution | Gros ordres, minimiser impact |
| `StandardDeviationExecutionModel` | Ordres Limit bases sur std | Reduire le slippage |
| `NullExecutionModel` | Ne fait rien | Desactive l'execution |

In [ ]:
# ImmediateExecutionModel - Le plus simple
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class ImmediateExecutionExample(QCAlgorithm):
    """
    Utilisation du ImmediateExecutionModel.
    Execute immediatement avec des ordres Market.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        symbols = ["SPY", "QQQ", "IWM", "TLT"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # === IMMEDIATE EXECUTION MODEL ===
        # Execute immediatement tous les targets avec des Market Orders
        self.SetExecution(ImmediateExecutionModel())
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=7))
        )
        
        self.Log("ImmediateExecutionModel configure")

print("ImmediateExecutionExample defini")
print("\nCaracteristiques ImmediateExecutionModel:")
print("  - Ordres Market executes immediatement")
print("  - Simple et rapide")
print("  - Pas de controle sur le prix d'execution")
print("  - Peut subir du slippage sur gros volumes")

In [ ]:
# VolumeWeightedAveragePriceExecutionModel (VWAP)
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class VWAPExecutionExample(QCAlgorithm):
    """
    Utilisation du VolumeWeightedAveragePriceExecutionModel.
    Split les ordres sur la journee pour suivre le VWAP.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        symbols = ["SPY", "QQQ", "IWM"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Minute)  # Resolution Minute requise
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # === VWAP EXECUTION MODEL ===
        # Split les ordres sur la periode pour suivre le VWAP
        self.SetExecution(
            VolumeWeightedAveragePriceExecutionModel()
        )
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=7))
        )
        
        self.Log("VolumeWeightedAveragePriceExecutionModel configure")

print("VWAPExecutionExample defini")
print("\nCaracteristiques VWAPExecutionModel:")
print("  - Decoupe les gros ordres en petites tranches")
print("  - Execute proportionnellement au volume du marche")
print("  - Minimise l'impact de marche")
print("  - Necessite Resolution.Minute ou plus fin")

In [ ]:
# StandardDeviationExecutionModel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class StdDevExecutionExample(QCAlgorithm):
    """
    Utilisation du StandardDeviationExecutionModel.
    Place des ordres Limit bases sur les deviations standard.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        symbols = ["SPY", "QQQ", "IWM"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Minute)
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # === STANDARD DEVIATION EXECUTION ===
        # Place des Limit Orders a price +/- N*std
        self.SetExecution(
            StandardDeviationExecutionModel(
                period=60,                # 60 periodes pour calculer std
                deviations=2              # 2 deviations standard
            )
        )
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=7))
        )
        
        self.Log("StandardDeviationExecutionModel configure")

print("StdDevExecutionExample defini")
print("\nCaracteristiques StandardDeviationExecutionModel:")
print("  - Place des Limit Orders")
print("  - Prix Limit = Price - N * StdDev (pour achats)")
print("  - Attend des conditions favorables")
print("  - Peut ne pas etre execute si le marche s'eloigne")

### Comparaison des Execution Models

| Modele | Vitesse | Controle Prix | Impact Marche | Complexite |
|--------|---------|---------------|---------------|------------|
| **Immediate** | Rapide | Aucun | Eleve | Simple |
| **VWAP** | Lent | Moyen | Faible | Moyenne |
| **StdDev** | Variable | Eleve | Faible | Moyenne |

---

## Partie 7 : Custom Smart Execution Model (12 min)

### Objectif

Creer un **Execution Model personnalise** qui :

1. Verifie le **spread** avant d'executer
2. Utilise **Market Orders** si le spread est acceptable
3. Utilise **Limit Orders** si le spread est trop large
4. Controle le **slippage maximum**

In [ ]:
# Smart Execution Model avec controle du spread
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class SmartExecutionModel(ExecutionModel):
    """
    Execution Model intelligent qui adapte le type d'ordre
    selon le spread bid-ask.
    """
    
    def __init__(self, max_slippage_percent=0.001):
        """
        Args:
            max_slippage_percent: Slippage maximum acceptable (0.001 = 0.1%)
        """
        self.max_slippage = max_slippage_percent
        self.targets_collection = PortfolioTargetCollection()
    
    def Execute(self, algorithm, targets):
        """
        Execute les PortfolioTargets.
        
        Args:
            algorithm: QCAlgorithm
            targets: IEnumerable[PortfolioTarget]
        """
        # Mettre a jour la collection de targets
        self.targets_collection.AddRange(targets)
        
        for target in self.targets_collection:
            symbol = target.Symbol
            
            # Verifier que le security existe
            if symbol not in algorithm.Securities:
                continue
            
            security = algorithm.Securities[symbol]
            
            # Calculer la quantite a trader
            current_qty = algorithm.Portfolio[symbol].Quantity
            
            # Calculer target quantity depuis le poids
            target_value = algorithm.Portfolio.TotalPortfolioValue * target.Quantity
            target_qty = int(target_value / security.Price) if security.Price > 0 else 0
            
            delta = target_qty - current_qty
            
            # Ne rien faire si pas de changement significatif
            if abs(delta) < 1:
                continue
            
            # Calculer le spread
            bid_price = security.BidPrice
            ask_price = security.AskPrice
            
            if bid_price <= 0 or ask_price <= 0:
                # Pas de quote disponible -> Market Order
                algorithm.MarketOrder(symbol, delta)
                algorithm.Debug(f"{symbol.Value}: Market Order (no quotes) - {delta} shares")
                continue
            
            spread = ask_price - bid_price
            spread_pct = spread / security.Price
            
            algorithm.Debug(f"{symbol.Value}: Spread = {spread_pct:.4%} (max: {self.max_slippage:.4%})")
            
            # Decision: Market ou Limit?
            if spread_pct < self.max_slippage * 2:
                # Spread acceptable -> Market Order
                algorithm.MarketOrder(symbol, delta)
                algorithm.Debug(f"{symbol.Value}: Market Order - {delta} shares")
            else:
                # Spread trop large -> Limit Order
                if delta > 0:  # Achat
                    limit_price = bid_price + spread * 0.3  # Bid + 30% du spread
                else:  # Vente
                    limit_price = ask_price - spread * 0.3  # Ask - 30% du spread
                
                algorithm.LimitOrder(symbol, delta, limit_price)
                algorithm.Debug(f"{symbol.Value}: Limit Order @ ${limit_price:.2f} - {delta} shares")
        
        # Nettoyer les targets executes
        self.targets_collection.ClearFulfilled(algorithm)

print("SmartExecutionModel defini")
print("\nLogique:")
print("  1. Calcule le spread bid-ask")
print("  2. Si spread < 2x max_slippage -> Market Order")
print("  3. Sinon -> Limit Order a prix avantageux")
print("  4. Reduit le slippage sur marches peu liquides")

In [ ]:
# Utilisation du SmartExecutionModel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class SmartExecutionAlgorithm(QCAlgorithm):
    """
    Algorithme utilisant le SmartExecutionModel.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs (certains plus liquides que d'autres)
        symbols = ["SPY", "QQQ", "IWM", "VNQ", "EEM"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Minute)
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # === SMART EXECUTION MODEL ===
        self.SetExecution(
            SmartExecutionModel(max_slippage_percent=0.001)  # Max 0.1% slippage
        )
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=7))
        )
        
        self.Log("SmartExecutionAlgorithm initialized")
        self.Log("  Max slippage: 0.1%")

print("SmartExecutionAlgorithm defini")
print("\nAvantages:")
print("  - Adapte le type d'ordre au marche")
print("  - Reduit les couts de transaction")
print("  - Plus efficace sur actifs moins liquides")

---

## Partie 8 : Risk Management Models (15 min)

### Role du Risk Management Model

Le **Risk Management Model** intercepte les PortfolioTargets et les ajuste selon des regles de risque. Il peut :

- **Reduire** les positions qui depassent un seuil
- **Liquider** les positions avec trop de pertes
- **Limiter** l'exposition par secteur

### Risk Models integres

| Modele | Description |
|--------|-------------|
| `MaximumDrawdownPercentPerSecurity` | Liquide si drawdown > X% par position |
| `MaximumUnrealizedProfitPercentPerSecurity` | Take profit si gain > X% |
| `MaximumSectorExposureRiskManagementModel` | Limite l'exposition par secteur |
| `TrailingStopRiskManagementModel` | Trailing stop automatique |
| `NullRiskManagementModel` | Desactive le risk management |

In [ ]:
# MaximumDrawdownPercentPerSecurity
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class DrawdownRiskExample(QCAlgorithm):
    """
    Exemple avec MaximumDrawdownPercentPerSecurity.
    Liquide automatiquement si perte > 5% sur une position.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        symbols = ["SPY", "QQQ", "AAPL", "MSFT", "TSLA"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        
        # === RISK MANAGEMENT: MAX DRAWDOWN 5% ===
        self.SetRiskManagement(
            MaximumDrawdownPercentPerSecurity(0.05)  # Liquide si perte > 5%
        )
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=30))
        )
        
        self.Log("MaximumDrawdownPercentPerSecurity(0.05) configure")

print("DrawdownRiskExample defini")
print("\nFonctionnement:")
print("  - Surveille le drawdown de chaque position")
print("  - Si une position perd > 5% depuis son max -> liquidation")
print("  - Protection automatique contre les grosses pertes")

In [ ]:
# MaximumSectorExposureRiskManagementModel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class SectorExposureRiskExample(QCAlgorithm):
    """
    Exemple avec MaximumSectorExposureRiskManagementModel.
    Limite l'exposition par secteur a 20%.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Mix de secteurs
        symbols = ["AAPL", "MSFT", "GOOGL",  # Tech
                   "JPM", "BAC",              # Finance
                   "JNJ", "PFE",              # Healthcare
                   "XOM", "CVX"]              # Energy
        
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        
        # === RISK MANAGEMENT: MAX 20% PAR SECTEUR ===
        self.SetRiskManagement(
            MaximumSectorExposureRiskManagementModel(0.20)  # Max 20% par secteur
        )
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=30))
        )
        
        self.Log("MaximumSectorExposureRiskManagementModel(0.20) configure")

print("SectorExposureRiskExample defini")
print("\nFonctionnement:")
print("  - Classifie les positions par secteur")
print("  - Si un secteur depasse 20% -> reduit les positions")
print("  - Assure la diversification sectorielle")

In [ ]:
# CompositeRiskManagementModel - Combiner plusieurs risk models
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class CompositeRiskExample(QCAlgorithm):
    """
    Exemple avec CompositeRiskManagementModel.
    Combine plusieurs risk models.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        symbols = ["SPY", "QQQ", "AAPL", "MSFT", "GOOGL", "TSLA", "JPM", "JNJ"]
        for symbol in symbols:
            self.AddEquity(symbol, Resolution.Daily)
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        
        # === COMPOSITE RISK MANAGEMENT ===
        # Combine plusieurs regles de risque
        self.SetRiskManagement(
            CompositeRiskManagementModel(
                MaximumDrawdownPercentPerSecurity(0.05),     # Stop loss 5%
                MaximumSectorExposureRiskManagementModel(0.20)  # Max 20% par secteur
            )
        )
        
        self.SetAlpha(
            ConstantAlphaModel(InsightType.Price, InsightDirection.Up, timedelta(days=30))
        )
        
        self.Log("CompositeRiskManagementModel configure:")
        self.Log("  - MaximumDrawdownPercentPerSecurity(0.05)")
        self.Log("  - MaximumSectorExposureRiskManagementModel(0.20)")

print("CompositeRiskExample defini")
print("\nAvantages du CompositeRiskManagementModel:")
print("  - Applique plusieurs regles simultanement")
print("  - Chaque regle est evaluee independamment")
print("  - Protection multi-niveaux")

### Resume des Risk Management Models

| Modele | Parametre | Effet |
|--------|-----------|-------|
| `MaximumDrawdownPercentPerSecurity(0.05)` | 5% | Liquide si perte > 5% |
| `MaximumUnrealizedProfitPercentPerSecurity(0.10)` | 10% | Take profit si gain > 10% |
| `MaximumSectorExposureRiskManagementModel(0.20)` | 20% | Max 20% par secteur |
| `TrailingStopRiskManagementModel(0.03)` | 3% | Trailing stop 3% |
| `CompositeRiskManagementModel(...)` | Multiple | Combine plusieurs models |

---

## Partie 9 : Strategie Framework Complete (20 min)

### Objectif

Construire une **strategie complete** utilisant tous les composants du Framework :

1. **Universe Selection** : Fine Fundamental (Volume + Market Cap)
2. **Alpha Model** : Momentum avec magnitude et confidence
3. **Portfolio Construction** : Risk Parity
4. **Execution** : Smart Execution avec controle du spread
5. **Risk Management** : Composite (Drawdown + Sector)

In [ ]:
# Strategie Framework Complete
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
import numpy as np

class FullFrameworkAlgorithm(QCAlgorithm):
    """
    Strategie complete utilisant tous les composants du Framework:
    
    1. Universe: Coarse + Fine (Large Cap, High Volume)
    2. Alpha: Momentum avec magnitude/confidence
    3. Portfolio: Risk Parity (volatilite inverse)
    4. Execution: Smart (spread-aware)
    5. Risk: Composite (Drawdown + Sector)
    """
    
    def Initialize(self):
        # === CONFIGURATION ===
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe settings
        self.UniverseSettings.Resolution = Resolution.Daily
        self.num_stocks = 30
        
        # === 1. UNIVERSE SELECTION ===
        self.AddUniverse(self.CoarseFilter, self.FineFilter)
        
        # === 2. ALPHA MODEL ===
        self.SetAlpha(MomentumAlphaModelFramework(lookback=60))
        
        # === 3. PORTFOLIO CONSTRUCTION ===
        self.SetPortfolioConstruction(
            RiskParityPortfolioConstructionFramework(rebalance_period=30, lookback=60)
        )
        
        # === 4. EXECUTION ===
        self.SetExecution(
            SmartExecutionModelFramework(max_slippage_percent=0.001)
        )
        
        # === 5. RISK MANAGEMENT ===
        self.SetRiskManagement(
            CompositeRiskManagementModel(
                MaximumDrawdownPercentPerSecurity(0.05),
                MaximumSectorExposureRiskManagementModel(0.25)
            )
        )
        
        self.Log("="*60)
        self.Log("FULL FRAMEWORK ALGORITHM INITIALIZED")
        self.Log("="*60)
        self.Log("Universe: Coarse + Fine (Top 30 by Market Cap)")
        self.Log("Alpha: Momentum (60 days)")
        self.Log("Portfolio: Risk Parity")
        self.Log("Execution: Smart (spread-aware)")
        self.Log("Risk: Max 5% DD per security, Max 25% sector exposure")
        self.Log("="*60)
    
    # === UNIVERSE SELECTION ===
    def CoarseFilter(self, coarse):
        """Premier filtre: liquidite et donnees fondamentales."""
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > 10
                    and x.DollarVolume > 10000000]
        
        sorted_by_volume = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        return [x.Symbol for x in sorted_by_volume[:200]]  # 200 candidats
    
    def FineFilter(self, fine):
        """Deuxieme filtre: Market Cap."""
        filtered = [x for x in fine if x.MarketCap > 5e9]  # > $5B Market Cap
        sorted_by_cap = sorted(filtered, key=lambda x: x.MarketCap, reverse=True)
        
        selected = sorted_by_cap[:self.num_stocks]
        self.Log(f"Universe: {len(selected)} stocks selected")
        
        return [x.Symbol for x in selected]
    
    def OnData(self, data):
        """Le Framework gere tout automatiquement."""
        pass
    
    def OnEndOfAlgorithm(self):
        """Resume final."""
        self.Log("\n" + "="*60)
        self.Log("FINAL SUMMARY")
        self.Log("="*60)
        self.Log(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        total_return = (self.Portfolio.TotalPortfolioValue / self.StartingCash - 1) * 100
        self.Log(f"Total Return: {total_return:.2f}%")
        self.Log("="*60)


# === ALPHA MODEL ===
class MomentumAlphaModelFramework(AlphaModel):
    """Alpha Model basé sur le momentum."""
    
    def __init__(self, lookback=60):
        self.lookback = lookback
        self.securities = []
    
    def Update(self, algorithm, data):
        insights = []
        
        for symbol in self.securities:
            if not data.ContainsKey(symbol):
                continue
            
            history = algorithm.History(symbol, self.lookback, Resolution.Daily)
            if history.empty or len(history) < self.lookback * 0.8:
                continue
            
            try:
                returns = history['close'].pct_change().dropna()
                momentum = returns.mean()
                volatility = returns.std()
                
                if volatility == 0:
                    continue
                
                direction = InsightDirection.Up if momentum > 0 else InsightDirection.Down
                magnitude = min(abs(momentum) * 10, 0.2)
                sharpe = momentum / volatility
                confidence = min(max(abs(sharpe) / 3, 0.1), 1.0)
                
                insight = Insight.Price(symbol, timedelta(days=30), direction, magnitude, confidence)
                insights.append(insight)
            except:
                continue
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.AddedSecurities:
            if security.Symbol not in self.securities:
                self.securities.append(security.Symbol)
        for security in changes.RemovedSecurities:
            if security.Symbol in self.securities:
                self.securities.remove(security.Symbol)


# === PORTFOLIO CONSTRUCTION ===
class RiskParityPortfolioConstructionFramework(PortfolioConstructionModel):
    """Portfolio Construction basé sur Risk Parity."""
    
    def __init__(self, rebalance_period=30, lookback=60):
        self.rebalance_period = rebalance_period
        self.lookback = lookback
        self.last_rebalance = datetime.min
    
    def CreateTargets(self, algorithm, insights):
        targets = []
        
        if (algorithm.Time - self.last_rebalance).days < self.rebalance_period:
            return targets
        
        self.last_rebalance = algorithm.Time
        active_insights = [i for i in insights if i.Direction != InsightDirection.Flat]
        
        if len(active_insights) == 0:
            return targets
        
        volatilities = {}
        for insight in active_insights:
            symbol = insight.Symbol
            history = algorithm.History(symbol, self.lookback, Resolution.Daily)
            
            if history.empty or len(history) < self.lookback * 0.8:
                continue
            
            try:
                returns = history['close'].pct_change().dropna()
                vol = returns.std() * np.sqrt(252)
                if vol > 0:
                    volatilities[symbol] = vol
            except:
                continue
        
        if len(volatilities) == 0:
            return targets
        
        inv_vols = {s: 1.0 / v for s, v in volatilities.items()}
        total_inv_vol = sum(inv_vols.values())
        
        for insight in active_insights:
            symbol = insight.Symbol
            if symbol not in inv_vols:
                continue
            
            base_weight = inv_vols[symbol] / total_inv_vol
            direction = 1 if insight.Direction == InsightDirection.Up else -1
            final_weight = direction * base_weight
            
            targets.append(PortfolioTarget(symbol, final_weight))
        
        algorithm.Log(f"Risk Parity: {len(targets)} targets")
        return targets
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.RemovedSecurities:
            if security.Invested:
                algorithm.Liquidate(security.Symbol)


# === EXECUTION MODEL ===
class SmartExecutionModelFramework(ExecutionModel):
    """Execution Model intelligent avec controle du spread."""
    
    def __init__(self, max_slippage_percent=0.001):
        self.max_slippage = max_slippage_percent
        self.targets_collection = PortfolioTargetCollection()
    
    def Execute(self, algorithm, targets):
        self.targets_collection.AddRange(targets)
        
        for target in self.targets_collection:
            symbol = target.Symbol
            
            if symbol not in algorithm.Securities:
                continue
            
            security = algorithm.Securities[symbol]
            current_qty = algorithm.Portfolio[symbol].Quantity
            target_value = algorithm.Portfolio.TotalPortfolioValue * target.Quantity
            target_qty = int(target_value / security.Price) if security.Price > 0 else 0
            delta = target_qty - current_qty
            
            if abs(delta) < 1:
                continue
            
            bid_price = security.BidPrice
            ask_price = security.AskPrice
            
            if bid_price <= 0 or ask_price <= 0:
                algorithm.MarketOrder(symbol, delta)
                continue
            
            spread_pct = (ask_price - bid_price) / security.Price
            
            if spread_pct < self.max_slippage * 2:
                algorithm.MarketOrder(symbol, delta)
            else:
                spread = ask_price - bid_price
                limit_price = bid_price + spread * 0.3 if delta > 0 else ask_price - spread * 0.3
                algorithm.LimitOrder(symbol, delta, limit_price)
        
        self.targets_collection.ClearFulfilled(algorithm)


print("FullFrameworkAlgorithm defini avec tous les composants")
print("\nComposants:")
print("  1. Universe: CoarseFilter + FineFilter")
print("  2. Alpha: MomentumAlphaModelFramework")
print("  3. Portfolio: RiskParityPortfolioConstructionFramework")
print("  4. Execution: SmartExecutionModelFramework")
print("  5. Risk: CompositeRiskManagementModel")

### Architecture de la strategie complete

```
                    FULL FRAMEWORK ALGORITHM
                    ========================

    +------------------+
    | Universe Filter  |     Coarse: Vol > $10M, Price > $10
    | (Coarse + Fine)  | --> Fine: Market Cap > $5B
    +--------+---------+     -> 30 stocks
             |
             v
    +------------------+
    |  Alpha Model     |     Momentum 60 jours
    | (Momentum)       | --> Direction, Magnitude, Confidence
    +--------+---------+     -> Insights
             |
             v
    +------------------+
    |Portfolio Constr. |     Risk Parity (1/volatilite)
    | (Risk Parity)    | --> Poids equalises en risque
    +--------+---------+     -> Portfolio Targets
             |
             v
    +------------------+
    | Risk Management  |     Max DD 5%, Max Sector 25%
    | (Composite)      | --> Ajuste les targets
    +--------+---------+     -> Targets ajustes
             |
             v
    +------------------+
    |   Execution      |     Market si spread OK
    | (Smart)          | --> Limit si spread large
    +------------------+     -> Ordres
```

---

## Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons appris a :

| Composant | Modeles | Points Cles |
|-----------|---------|-------------|
| **Portfolio Construction** | EqualWeighting, InsightWeighting, MeanVariance, Custom | Transforme Insights -> Targets |
| **Execution** | Immediate, VWAP, StdDev, Smart Custom | Gere le type d'ordre et le timing |
| **Risk Management** | MaxDrawdown, SectorExposure, Composite | Protege le portefeuille |

### Tableau de choix des modeles

| Situation | Portfolio Model | Execution Model | Risk Model |
|-----------|-----------------|-----------------|------------|
| **Simple, liquide** | EqualWeighting | Immediate | MaxDrawdown |
| **Signaux sophistiques** | InsightWeighting | Immediate | Composite |
| **Optimisation risque** | MeanVariance | VWAP | SectorExposure |
| **Multi-asset class** | Risk Parity | Smart | Composite |

### Points Cles a Retenir

1. **Separation des responsabilites** : Chaque modele a un role precis
2. **Modularite** : Facile de changer un composant sans affecter les autres
3. **Custom models** : Heriter des classes de base pour implementer votre logique
4. **CompositeRiskModel** : Combiner plusieurs regles de risque
5. **Smart Execution** : Adapter le type d'ordre au marche

### Ressources Complementaires

- [Portfolio Construction Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/portfolio-construction)
- [Execution Model Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/execution)
- [Risk Management Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/risk-management)
- [Framework Overview](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/overview)

---

**Notebook complete. Vous maitrisez maintenant les composants avances du Framework QuantConnect.**